# Create an InterActive Graph for August 2020

In [7]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from dateutil.parser import parse
from datetime import date
import datetime
import re
from datetime import timedelta
import openpyxl
#from sklearn.metrics import mean_squared_error
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# Create figure with secondary y-axis

In [ ]:
wb1 = openpyxl.load_workbook(r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\8) August _2020.xlsx', data_only=True)

#----Importing the Pi data for MS flow, rec gas, feed temp

df_MSPi = pd.read_excel(r'C:\Users\v.t.flores\Documents\MS Flow Aug.xlsx', sheet_name='PiData_MS FLOW', index_col=False)

In [8]:
#--------------------------------Functions and Classes------------------------------------------------------------------------
#
#used in the analysis results data. cleans up the sampling time entry
def time_cleaner(x):
    time_regex = re.compile(r'[0-8a-zA-Z:]', re.IGNORECASE| re.VERBOSE|re.DOTALL)
    timeact = time_regex.findall(x)
    newtime = ''.join(timeact)
    t = parse(x)
    new_str_time = str(t.hour)+':'+str(t.minute)+':'+str(t.second)
    return new_str_time

def generate_df_from_analysis(workbookTab, col_sampleDate, col_sampleTime, col_analysis, col_title):
    #get the start and end sample dates
    Smpl_day_index = list()
    Chem_Element_datelist = list()
    Chem_Element_list = list()
    Smpl_day_index = list()
    Chem_Element_datelist = list()
    Chem_Element_list = list()
    plus_oneday_list = ['0:0:0', '1:0:0', '2:0:0', '3:0:0', '4:0:0', '5:0:0']
    #gets the date day from the B column of the analysis file if there is value in the cell
    for i in range(1, workbookTab.max_row):
        if type(workbookTab[str(col_sampleDate) + str(i)].value) == int or type(workbookTab[str(col_sampleDate) + str(i)].value) == float:
            Smpl_day_index.append(i)
    #get the ending row that has date day
    for j in range(1, workbookTab.max_row):
        if type(workbookTab[str(col_sampleTime) + str(j)].value) == datetime.datetime:
            smpl_end_index = j
    for y in range(0, len(Smpl_day_index)):
        if Smpl_day_index[y] != Smpl_day_index[-1]:      
            for x in range(Smpl_day_index[y], Smpl_day_index[y+1]):
                if workbookTab[str(col_analysis)+str(x)].value is None: 
                    pass
                else:
                    if time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value)) in plus_oneday_list:
                        date = str((currentmonthyr)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                        +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                        Chem_Element_datelist.append(parse(date))  
                        chem_element = workbookTab[str(col_analysis)+str(x)].value
                        Chem_Element_list.append(chem_element)
                    else:
                        date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                        +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                        Chem_Element_datelist.append(parse(date))  
                        chem_element = workbookTab[str(col_analysis)+str(x)].value
                        Chem_Element_list.append(chem_element)

        if Smpl_day_index[y] == Smpl_day_index[-1]:
            for x in range(Smpl_day_index[y], smpl_end_index+1):
                if workbookTab[str(col_analysis)+str(x)].value is None:   
                    pass
                else:
                    if time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value)) in plus_oneday_list:
                        date = str((currentmonthyr)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                        +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                        Chem_Element_datelist.append(parse(date))
                        chem_element = workbookTab[str(col_analysis)+str(x)].value
                        Chem_Element_list.append(chem_element)
                    else:
                        date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                        +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                        Chem_Element_datelist.append(parse(date))
                        chem_element = workbookTab[str(col_analysis)+str(x)].value
                        Chem_Element_list.append(chem_element)

    Chem_Element_list_pair = list(zip(Chem_Element_list, Chem_Element_datelist[0:]))
    Chem_Element_df = pd.DataFrame(Chem_Element_list_pair)
    Chem_Element_df.columns = [col_title, 'DATETIME'] ##check if list title is updated
    Chem_Element_df.set_index('DATETIME', inplace=True)

    return Chem_Element_df

def generate_add_trace(figname, df_elemental, axis_bool):
    colname = df_elemental.columns[0]
    figname.add_trace(go.Scatter(x=df_elemental.index, y=df_elemental[colname], name=colname), secondary_y=axis_bool,)
#---------------------------------------
class LabResults():
    def __init__(self, monthyear, filePath):
        #____monthyear format examle is datetime.date(2020,7,1)
        #____filepath example is '\\thgobna001\userdata\THPAL\...\ANALYSIS RESULTS 2020\7) July _2020.xlsx'
        #____add r' at the start of the filepath to apply the regex that allows the use of the filepath as raw
        #____sef.workbook is the uploaded excel workbook of the entire analysis files so it takes time to load
        #____Create an instance of the workbook by giving it a varible name with the sample format: Jan_2020
        
        import openpyxl
        import io
        with open(filePath, "rb") as f:
            in_mem_file = io.BytesIO(f.read())
        
        self.workbook = openpyxl.load_workbook(in_mem_file, data_only=True)
        self.monthyear = monthyear   
        
    def generate_df_from_analysis(self, tabname, list_lab_index):
        #____tabname is a string type name of the tab in the laboratory analysis excel file.
        #____-------Example tabnames: '104PU01', '105TK03', '106TK01'
        #____-------Inside these tabs are the specific analysis results done on the sample taken from the 'tabname' area .
        #____list_lab_index is a list with the sample format is: ['B', 'C', 'E', '104_Pb']
        #____-------the first letter 'B' is the column letter where the day of the month is written as integers 1,2,3..30
        #____-------the second letter 'C' is the column letter where the sampling time is written; e.g. '7:00:00 AM'
        #____-------the third letter 'E' is the column where the results of the specific analysis are listed
        #____-------the last item in the list is the string you'll use as the new title; e.g. '104_Pb'
        
        workbookTab = self.workbook[tabname]
        self.list_lab_index = list_lab_index
        col_sampleDate = self.list_lab_index[0]
        col_sampleTime = self.list_lab_index[1]
        col_analysis = self.list_lab_index[2]
        col_title = self.list_lab_index[3]
        currentmonthyr = self.monthyear
        
        #get the start and end sample dates
        Smpl_day_index = list()
        Chem_Element_datelist = list()
        Chem_Element_list = list()
        Smpl_day_index = list()
        Chem_Element_datelist = list()
        Chem_Element_list = list()
        plus_oneday_list = ['0:0:0', '1:0:0', '2:0:0', '3:0:0', '4:0:0', '5:0:0']
        #gets the date day from the B column of the analysis file if there is value in the cell
        for i in range(1, workbookTab.max_row):
            if type(workbookTab[str(col_sampleDate) + str(i)].value) == int or type(workbookTab[str(col_sampleDate) + str(i)].value) == float:
                Smpl_day_index.append(i)
        #get the ending row that has date day
        for j in range(1, workbookTab.max_row):
            if type(workbookTab[str(col_sampleTime) + str(j)].value) == datetime.datetime:
                smpl_end_index = j
        for y in range(0, len(Smpl_day_index)):
            if Smpl_day_index[y] != Smpl_day_index[-1]:      
                for x in range(Smpl_day_index[y], Smpl_day_index[y+1]):
                    if workbookTab[str(col_analysis)+str(x)].value is None: 
                        pass
                    else:
                        if time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value)) in plus_oneday_list:
                            date = str((currentmonthyr)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                            +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                            Chem_Element_datelist.append(parse(date))  
                            chem_element = workbookTab[str(col_analysis)+str(x)].value
                            Chem_Element_list.append(chem_element)
                        else:
                            date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                            +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                            Chem_Element_datelist.append(parse(date))  
                            chem_element = workbookTab[str(col_analysis)+str(x)].value
                            Chem_Element_list.append(chem_element)

            if Smpl_day_index[y] == Smpl_day_index[-1]:
                for x in range(Smpl_day_index[y], smpl_end_index+1):
                    if workbookTab[str(col_analysis)+str(x)].value is None:   
                        pass
                    else:
                        if time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value)) in plus_oneday_list:
                            date = str((currentmonthyr)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                            +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                            Chem_Element_datelist.append(parse(date))
                            chem_element = workbookTab[str(col_analysis)+str(x)].value
                            Chem_Element_list.append(chem_element)
                        else:
                            date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                            +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                            Chem_Element_datelist.append(parse(date))
                            chem_element = workbookTab[str(col_analysis)+str(x)].value
                            Chem_Element_list.append(chem_element)

        Chem_Element_list_pair = list(zip(Chem_Element_list, Chem_Element_datelist[0:]))
        Chem_Element_df = pd.DataFrame(Chem_Element_list_pair)
        Chem_Element_df.columns = [col_title, 'DATETIME'] ##check if list title is updated
        Chem_Element_df.set_index('DATETIME', inplace=True)

        return Chem_Element_df
    
def join_DF(dfsList1):
    df = dfsList1[0]
    for i in range(1,len(dfsList1)):
        df = df.join(dfsList1[i], how='outer')
    return df

def append_monthsDF(dfsListmonthly): #takes the list of dataframes from monthly lab results of one sampling area
    df = dfsListmonthly[0]
    for i in range(1,len(dfsListmonthly)):
        df = df.append(dfsListmonthly[i], ignore_index=True)
    return df    
def generate_combined_df_by_month(month_WB, sample_area, analysis_type):
    df_List = []
    for i in analysis_type:
        df_analysis = month_WB.generate_df_from_analysis(sample_area, i)
        df_List.append(df_analysis)
    for i in df_List:
        df = join_DF(df_List)
    return df



#----------------------------------------------------End-----------------------------------------------------------------
#
#  make_df_for_each_FT01_by_month_then_export_excel: This function exports the 
def make_df_for_each_FT01_by_month(monthLabRes, analysis_type_FT01AtoF_list):
    #month_ = LabResults(currentmonthyr_list[monthNumber-1], filepath_list[monthNumber-1])
    letters = 'ABCDEF'
    FT01_DF_list_by_filter = []
    for i in range(0,6):
        FT01_ = generate_combined_df_by_month(monthLabRes, '105FT01', analysis_type_FT01AtoF_list[i])
        FT01_.columns = [letters[i]+'_Zn',
                         letters[i]+'_ntu',
                         letters[i]+'_Zn_dr']
        FT01_DF_list_by_filter.append(FT01_)
    return join_DF(FT01_DF_list_by_filter)    

#--------------------------------------------------End-------------------------------------------------------------------
#
#
def generate_plot_from_df(figname, df,df_axis_truefalse_list):
    for i in range(0, len(df.columns)):
        colname = df.columns[i]
        figname.add_trace(go.Scatter(x=df[colname].index, y=df[colname], name=colname), secondary_y=df_axis_truefalse_list[i],)
#
#
#--------------------------------------------------------------End-----------------------------------------------------------------------
#-------------------------------------------------------------------------------------------------------------------    
#
# Analysis Results workbook per area
#
NTRL_lab = wb1['104PU01']
DeZn_TK05 = wb1['105TK05']
MS_lab = wb1['106TK01']
MS_VE02_lab = wb1['106VE02']
MS_VE04_lab = wb1['106VE04']

#--------Lists--------------------------------------------------------------------------------------------
# monthyear
currentmonthyr_01 = datetime.date(2020,1,1)
currentmonthyr_02 = datetime.date(2020,2,1)
currentmonthyr_03 = datetime.date(2020,3,1)
currentmonthyr_04 = datetime.date(2020,4,1)
currentmonthyr_05 = datetime.date(2020,5,1)
currentmonthyr_06 = datetime.date(2020,6,1)
currentmonthyr_07 = datetime.date(2020,7,1)
currentmonthyr_08 = datetime.date(2020,8,1)

#filepath
filepath_01 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\JANUARY 2020\1) JANUARY_2020.xlsx'
filepath_02 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\FEBRUARY 2020\2) FEBRUARY _2020.xlsx'
filepath_03 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\MARCH 2020\3) March _2020.xlsx'
filepath_04 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\APRIL 2020\4) April _2020.xlsx'
filepath_05 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\MAY 2020\5) MAY_2020.xlsx'
filepath_06 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\JUNE 2020\6) June _2020.xlsx'
filepath_07 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\JULY 2020\7) July _2020.xlsx'
filepath_08 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\8) August _2020.xlsx'

#list_lab_index for ntrl
ntrl_Pb = ['B', 'C', 'E', '104_Pb']
ntr_Zn =  ['B', 'C', 'F', '104_Zn']
ntrl_Cu = ['B', 'C', 'G', '104_Cu']
ntrl_Ni = ['B', 'C', 'H', '104_Ni']
ntrl_Co = ['B', 'C', 'I', '104_Co']
ntrl_Mn = ['B', 'C', 'K', '104_Mn']
ntrl_Cr = ['B', 'C', 'L', '104_Cr']
ntrl_Ca = ['B', 'C', 'M', '104_Ca']
ntrl_Si = ['B', 'C', 'N', '104_Si']
ntrl_Al = ['B', 'C', 'O', '104_Al']
ntrl_Mg = ['B', 'C', 'P', '104_Mg']
ntrl_Fe2 = ['B', 'C', 'R', '104_Fe2']
ntrl_pH = ['B', 'C', 'BB', '104_pH60']
ntrl_ORP = ['B', 'C', 'AJ', '104_ORP']
ntrl_NTU = ['B', 'C', 'AN', '104_NTU']
ntrl_FT = ['B', 'C', 'AW', '104_FT']

#List_lab_index for 105TK05
DeZn_TK05_Pb = ['B', 'C', 'E', '105TK05_Pb']
DeZn_TK05_Zn =['B', 'C', 'F', '105TK05_Zn']
DeZn_TK05_Cu =['B', 'C', 'G', '105TK05_Cu']
DeZn_TK05_Ni =['B', 'C', 'H', '105TK05_Ni']
DeZn_TK05_Co =['B', 'C', 'I', '105TK05_Co']
DeZn_TK05_Fe2 =['B', 'C', 'J', '105TK05_Fe']
DeZn_TK05_Mn =['B', 'C', 'K', '105TK05_Mn']
DeZn_TK05_Cr =['B', 'C', 'L', '105TK05_Cr']
DeZn_TK05_Ca =['B', 'C', 'M', '105TK05_Ca']
DeZn_TK05_Si =['B', 'C', 'N', '105TK05_Si']
DeZn_TK05_Al =['B', 'C', 'O', '105TK05_Al']
DeZn_TK05_Mg =['B', 'C', 'P', '105TK05_Mg']
DeZn_TK05_pH =['B', 'C', 'AF', '105TK05_pH']
DeZn_TK05_ORP =['B', 'C', 'AG', '105TK05_ORP']
DeZn_TK05_PS90 =['B', 'C', 'AP', '105TK05_PS90']

#List_lab_index for 105TK03
DeZn_TK03_105TK03_Zn_df = ['B', 'C', 'D', '105TK03_Zn']
DeZn_TK03_105TK03_Zn_Change_Ratio_df = ['B', 'C', 'BB', '105TK03_Zn_Change_Ratio']
DeZn_TK03_105TK05_Zn_df = ['B', 'C', 'BD', '105TK05_Zn']
DeZn_TK03_106TK01_Zn_df = ['B', 'C', 'BE', '106TK01_Zn']
DeZn_TK03_106TK01_ZnNi_df = ['B', 'C', 'BF', '106TK01_Zn/Ni']
DeZn_TK03_106TK01_Zn_DropRate_df = ['B', 'C', 'BG', '106TK01_Zn_DropRate']
DeZn_TK03_106VE04_Znprcnt_df = ['B', 'C', 'BH', '106VE04_%Zn']
DeZn_TK03_106TH01_Znprcnt_df = ['B', 'C', 'BI', '106TH01_%Zn']
DeZn_TK03_104PU01_Ni_df = ['B', 'C', 'BJ', '104PU01_Ni']
DeZn_TK03_105TK05_Ni_df = ['B', 'C', 'BK', '105TK05_Ni']
DeZn_TK03_Ni_Loss_TK05_df = ['B', 'C', 'BL', 'Ni_Loss_TK05']


#- Creating the polishing filter list of cell column letters and analysis type-------------------------------
#
FT01A_Zn_list = ['B', 'C', 'D', 'Zn']
FT01B_Zn_list = ['B', 'C', 'E', 'Zn']
FT01C_Zn_list = ['B', 'C', 'F', 'Zn']
FT01D_Zn_list = ['B', 'C', 'G', 'Zn']
FT01E_Zn_list = ['B', 'C', 'H', 'Zn']
FT01F_Zn_list = ['B', 'C', 'I', 'Zn']

FT01A_ntu_list = ['B', 'C', 'J', 'ntu']
FT01B_ntu_list = ['B', 'C', 'K', 'ntu']
FT01C_ntu_list = ['B', 'C', 'L', 'ntu']
FT01D_ntu_list = ['B', 'C', 'M', 'ntu']
FT01E_ntu_list = ['B', 'C', 'N', 'ntu']
FT01F_ntu_list = ['B', 'C', 'O', 'ntu']

FT01A_ZDR_list = ['B', 'C', 'J', 'Zn_dr']
FT01B_ZDR_list = ['B', 'C', 'K', 'Zn_dr']
FT01C_ZDR_list = ['B', 'C', 'L', 'Zn_dr']
FT01D_ZDR_list = ['B', 'C', 'M', 'Zn_dr']
FT01E_ZDR_list = ['B', 'C', 'N', 'Zn_dr']
FT01F_ZDR_list = ['B', 'C', 'O', 'Zn_dr']

analysis_type_FT01A_list = [FT01A_Zn_list,FT01A_ntu_list,FT01A_ZDR_list]
analysis_type_FT01B_list = [FT01B_Zn_list,FT01B_ntu_list,FT01B_ZDR_list]
analysis_type_FT01C_list = [FT01C_Zn_list,FT01C_ntu_list,FT01C_ZDR_list]
analysis_type_FT01D_list = [FT01D_Zn_list,FT01D_ntu_list,FT01D_ZDR_list]
analysis_type_FT01E_list = [FT01E_Zn_list,FT01E_ntu_list,FT01E_ZDR_list]
analysis_type_FT01F_list = [FT01F_Zn_list,FT01F_ntu_list,FT01F_ZDR_list]

# generates a dataframe for each column in the Analysis excel file 104PU01
analysis_type_MS_TK01_Pb = ['B', 'C', 'E', '106TK01_Pb']
analysis_type_MS_TK01_Zn = ['B', 'C', 'F', '106TK01_Zn']
analysis_type_MS_TK01_Cu = ['B', 'C', 'G', '106TK01_Cu']
analysis_type_MS_TK01_Ni = ['B', 'C', 'H', '106TK01_Ni']
analysis_type_MS_TK01_Co = ['B', 'C', 'I', '106TK01_Co']
analysis_type_MS_TK01_Fe = ['B', 'C', 'J', '106TK01_Fe']
analysis_type_MS_TK01_Mn = ['B', 'C', 'K', '106TK01_Mn']
analysis_type_MS_TK01_Cr = ['B', 'C', 'L', '106TK01_Cr']
analysis_type_MS_TK01_Ca = ['B', 'C', 'M', '106TK01_Ca']
analysis_type_MS_TK01_Si = ['B', 'C', 'N', '106TK01_Si']
analysis_type_MS_TK01_Al = ['B', 'C', 'O', '106TK01_Al']
analysis_type_MS_TK01_Mg = ['B', 'C', 'P', '106TK01_Mg']
analysis_type_MS_TK01_pH = ['B', 'C', 'AH', '106TK01_pH']
analysis_type_MS_TK01_ORP = ['B', 'C', 'AI', '106_ORP']
analysis_type_MS_TK01_ZnNi = ['B', 'C', 'BC', '106_Zn/Ni']

# generates a dataframe for each column in the Analysis excel file 104PU01
MS_VE04_Ni_Soln = ['B', 'C', 'D', '106VE04_Ni_soln']
MS_VE04_Pb = ['B', 'C', 'S', '106VE04_Pb']
MS_VE04_Zn = ['B', 'C', 'T', '106VE04_Zn']
MS_VE04_Cu = ['B', 'C', 'U', '106VE04_Cu']
MS_VE04_Ni = ['B', 'C', 'V', '106VE04_Ni']
MS_VE04_Co = ['B', 'C', 'W', '106VE04_Co']
MS_VE04_Fe = ['B', 'C', 'X', '106VE04_Fe']
MS_VE04_Mn = ['B', 'C', 'Y', '106VE04_Mn']
MS_VE04_Cr = ['B', 'C', 'Z', '106VE04_Cr']
MS_VE04_Ca = ['B', 'C', 'AA', '106VE04_Ca']
MS_VE04_Si = ['B', 'C', 'AB', '106VE04_Si']
MS_VE04_Al = ['B', 'C', 'AC', '106VE04_Al']
MS_VE04_Mg = ['B', 'C', 'AD', '106VE04_Mg']
MS_VE04_S = ['B', 'C', 'AE', '106VE04_S']
MS_VE04_pH = ['B', 'C', 'AG', '106VE04_pH']
MS_VE04_ORP = ['B', 'C', 'AH', '106VE04_ORP']


#-----------------------------------------------------------------------------------------------------------------------------------------------


# Per sample area list
analysis_type_list_NTRL = [ntrl_Pb, ntr_Zn, ntrl_Cu,
                           ntrl_Ni, ntrl_Co, ntrl_Mn,
                           ntrl_Cr, ntrl_Ca, ntrl_Si, 
                           ntrl_Al, ntrl_Mg, ntrl_Fe2, 
                           ntrl_pH, ntrl_ORP, ntrl_NTU, 
                           ntrl_FT]
analysis_type_list_DeZnTK05 = [DeZn_TK05_Pb, DeZn_TK05_Zn, DeZn_TK05_Cu, 
                               DeZn_TK05_Ni, DeZn_TK05_Co, DeZn_TK05_Fe2, 
                               DeZn_TK05_Mn, DeZn_TK05_Cr, DeZn_TK05_Ca, 
                               DeZn_TK05_Si, DeZn_TK05_Al, DeZn_TK05_Mg,
                               DeZn_TK05_pH, DeZn_TK05_ORP,DeZn_TK05_PS90]
analysis_type_list_DeZnTK03 = [DeZn_TK03_105TK03_Zn_df, DeZn_TK03_105TK03_Zn_Change_Ratio_df, DeZn_TK03_105TK05_Zn_df,
                               DeZn_TK03_106TK01_Zn_df, DeZn_TK03_106TK01_ZnNi_df, DeZn_TK03_106TK01_Zn_DropRate_df,
                               DeZn_TK03_106VE04_Znprcnt_df, DeZn_TK03_106TH01_Znprcnt_df, DeZn_TK03_104PU01_Ni_df,
                               DeZn_TK03_105TK05_Ni_df, DeZn_TK03_Ni_Loss_TK05_df]
analysis_type_FT01AtoF_list = [analysis_type_FT01A_list,
                               analysis_type_FT01B_list,
                               analysis_type_FT01C_list,
                               analysis_type_FT01D_list,
                               analysis_type_FT01E_list,
                               analysis_type_FT01F_list]
analysis_type_MS_TK01 = [analysis_type_MS_TK01_Pb, analysis_type_MS_TK01_Zn, analysis_type_MS_TK01_Cu,
                         analysis_type_MS_TK01_Ni, analysis_type_MS_TK01_Co, analysis_type_MS_TK01_Fe,
                         analysis_type_MS_TK01_Mn, analysis_type_MS_TK01_Cr, analysis_type_MS_TK01_Ca,
                         analysis_type_MS_TK01_Si, analysis_type_MS_TK01_Al, analysis_type_MS_TK01_Mg,
                         analysis_type_MS_TK01_pH, analysis_type_MS_TK01_ORP, analysis_type_MS_TK01_ZnNi]

analysis_type_MS_VE04 = [MS_VE04_Ni_Soln, MS_VE04_Pb, MS_VE04_Zn,
                         MS_VE04_Cu, MS_VE04_Ni, MS_VE04_Co,
                         MS_VE04_Fe,MS_VE04_Mn,MS_VE04_Cr,
                         MS_VE04_Ca, MS_VE04_Si, MS_VE04_Al,
                         MS_VE04_Mg, MS_VE04_S, MS_VE04_pH,
                         MS_VE04_ORP]

# Per month datetime.datetime list
currentmonthyr_list = [currentmonthyr_01, currentmonthyr_02, currentmonthyr_03, currentmonthyr_04,
                       currentmonthyr_05, currentmonthyr_06, currentmonthyr_07, currentmonthyr_08]

# List of filepath for the analysis files file address
filepath_list = [filepath_01, filepath_02, filepath_03, filepath_04, filepath_05, filepath_06, filepath_07, filepath_08]



Aug_2020 = LabResults(currentmonthyr_08, filepath_08)
Aug2020_ntrl_df = generate_combined_df_by_month(Aug_2020, '104PU01', analysis_type_list_NTRL)
Aug2020_TK05_df = generate_combined_df_by_month(Aug_2020, '105TK05', analysis_type_list_DeZnTK05)
Aug2020_TK03_df = generate_combined_df_by_month(Aug_2020, '105TK03', analysis_type_list_DeZnTK03)
Aug2020_TK03_df = Aug2020_TK03_df.drop(['104PU01_Ni', '105TK05_Ni'], axis=1)
Aug2020_FT01AtoF_df = make_df_for_each_FT01_by_month(Aug_2020, analysis_type_FT01AtoF_list)                
Aug2020_MSTK01_df = generate_combined_df_by_month(Aug_2020, '106TK01', analysis_type_MS_TK01)    
Aug2020_MSVE04_df = generate_combined_df_by_month(Aug_2020, '106VE04', analysis_type_MS_VE04)    

MS_VE02_Ni_soln_df = generate_df_from_analysis(MS_VE02_lab, 'B', 'C', 'D', '106VE02_Ni')




#-------------------------------------------------------------------------------------------------------------------------#
#---------------------Code for predicting the 106VE04 Ni soln by applying the coefficients  from RidgeRegression----------#

#------------------Drop the row[0]. This is the title tag use in the pi datalink. We don't need it in pandas df.
#------------------What remain are the column title and the rest of the rows of data.

df_MSPi = df_MSPi.drop(df_MSPi.index[0])
#Step2: Parse the DATETIME column
df_MSPi['DATETIME'] = df_MSPi['DATETIME'].apply(lambda x: parse(str(x)))
df_MSPi.set_index('DATETIME', inplace=True)
df_MSPi = df_MSPi

# Step3: Convert each column to numeric to handle nan values and other comments like 'No data...'. Convering to numeric
#------ converts them into a NaN value that pandas understands.
for i in df_MSPi.columns:
    df_MSPi[i] = pd.to_numeric(df_MSPi[i], errors='coerce')

# Step4: Select which FT is online based on their flowrates. The one with the greater value is online.

df_MSPi['MS_Flow'] = np.where(df_MSPi['MS_FLOW_A']>df_MSPi['MS_FLOW_B'], df_MSPi['MS_FLOW_A'], df_MSPi['MS_FLOW_B'])

# Step5: Selec the rows for MS flow, feed temp, rec gas. Then put them in a single dataframe.
df_MSPi_1 = df_MSPi[['MS_Flow',
                     'MS_Feed_T',
                     'Rec_Gas',
                     'H2S_Flow',
                     'NaSH_Flow']]

#-------Get the columns for 106TK01 lab resulsts on Mg, pH, Ni, Fe
df_VE04_Ni_pred = Aug2020_MSTK01_df[['106TK01_Mg',
                                   '106TK01_pH',
                                   '106TK01_Ni',
                                   '106TK01_Fe']]

DF_lab = Aug2020_MSVE04_df[['106VE04_Ni_soln']].join(MS_VE02_Ni_soln_df, how='outer').join(df_VE04_Ni_pred, how='outer')

df_VE02_04_Ni_MSpi = df_MSPi_1.join(DF_lab, how='outer')
df_VE02_04_Ni_MSpi = df_VE02_04_Ni_MSpi.interpolate(method='linear', axis=0, limit_direction='forward')

# Step6: Create a dictionary for the regression coefficients.
# Load the ridgereg_intercept from excel
#
# Use this for reporting ve04 predictions for the current time: 
#                     df_intercept = pd.read_excel(r'C:\Users\v.t.flores\Documents\ridgereg_intercept_MS2020_VE04_Pred.xlsx', index_col=False)
#

#
#
#
#

DF = df_VE02_04_Ni_MSpi

DF['VE04_Ni_Soln_pred'] = DF['106TK01_Ni']*-0.00110484936518315 + \
                                      DF['106TK01_Fe']*-0.00648388453313587 + \
                                      DF['106TK01_Mg']*-0.00106452774044569 + \
                                      DF['106TK01_pH']*-0.00305238779183487 + \
                                      DF['MS_Flow']*-0.0000287630970770015 + \
                                      DF['MS_Feed_T']*-0.00046942105737559 + \
                                      DF['Rec_Gas']*-0.0000109520371644774+\
                                      DF['H2S_Flow']*0.000020559903888341 + \
                                      DF['NaSH_Flow']*0.00296198950205355 + \
                                      DF['106VE02_Ni']*0.332507448549008 + \
                                      0.0595791520699017
#
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------
df_VE04_Ni_Pred_solo = DF['VE04_Ni_Soln_pred']
# Step10: Adjust the predicted values for retention time by temporarily transfering it to a separate df, 
#         then adding 2 hrs to it's datetime. Then rejoining it to the df_VE04Ni_MSpi df.

df_VE04Ni_MSpi = DF.drop('VE04_Ni_Soln_pred', axis=1) # Drop the VE04_Ni_soln_pred from the main df. Join it after adding retention time

df_VE04_Ni_Pred_solo = df_VE04_Ni_Pred_solo.reset_index()
df_VE04_Ni_Pred_solo['DATETIME'] =  df_VE04_Ni_Pred_solo['DATETIME'] + datetime.timedelta(hours=0)
df_VE04_Ni_Pred_solo.set_index('DATETIME', inplace=True)

# Join the time adjusted VE04_Ni_pred_solo
df_VE04Ni_MSpi = df_VE04Ni_MSpi.join(df_VE04_Ni_Pred_solo, how='outer')
df_VE04Ni_MSpi['error'] = abs(df_VE04Ni_MSpi['106VE04_Ni_soln']-df_VE04Ni_MSpi['VE04_Ni_Soln_pred'])/df_VE04Ni_MSpi['106VE04_Ni_soln']


#-------------------------------------------End of VE04 Ni Prediction Code -------------------------------------------####

Aug2020_ntrl_df = Aug2020_ntrl_df.interpolate(method='linear', limit_direction='forward', axis=0)
Aug2020_ntrl_df.dropna(how='any', inplace=True)

Aug2020_TK03_df = Aug2020_TK03_df.interpolate(method='linear', limit_direction='forward', axis=0)
Aug2020_TK03_df.dropna(how='any', inplace=True)
Aug2020_TK05_df = Aug2020_TK05_df.interpolate(method='linear', limit_direction='forward', axis=0)
Aug2020_TK05_df.dropna(how='any', inplace=True)
Aug2020_MSTK01_df = Aug2020_MSTK01_df.interpolate(method='linear', limit_direction='forward', axis=0)
Aug2020_MSTK01_df.dropna(how='any', inplace=True)
Aug2020_MSVE04_df = Aug2020_MSVE04_df.interpolate(method='linear', limit_direction='forward', axis=0)
Aug2020_MSVE04_df.dropna(how='any', inplace=True)

#-------------------------------------------------------------------------------------------------------------------

#--------------------------------------------------------------------- Polishing Filter-----------------------------------------------------------------
A = Aug2020_FT01AtoF_df[Aug2020_FT01AtoF_df.columns[0:3]]
A['Filter'] = 'FT01A'
B = Aug2020_FT01AtoF_df[Aug2020_FT01AtoF_df.columns[3:6]]
B['Filter'] = 'FT01B'
C = Aug2020_FT01AtoF_df[Aug2020_FT01AtoF_df.columns[6:9]]
C['Filter'] = 'FT01C'
D = Aug2020_FT01AtoF_df[Aug2020_FT01AtoF_df.columns[9:12]]
D['Filter'] = 'FT01D'
E = Aug2020_FT01AtoF_df[Aug2020_FT01AtoF_df.columns[12:15]]
E['Filter'] = 'FT01E'
F = Aug2020_FT01AtoF_df[Aug2020_FT01AtoF_df.columns[15:18]]
F['Filter'] = 'FT01F'

A.reset_index(inplace=True)
B.reset_index(inplace=True)
C.reset_index(inplace=True)
D.reset_index(inplace=True)
E.reset_index(inplace=True)
F.reset_index(inplace=True)

A.columns = ['DATETIME', 'Zn', 'ntu', 'Zn_dr', 'Filter']
B.columns = ['DATETIME', 'Zn', 'ntu', 'Zn_dr', 'Filter']
C.columns = ['DATETIME', 'Zn', 'ntu', 'Zn_dr', 'Filter']
D.columns = ['DATETIME', 'Zn', 'ntu', 'Zn_dr', 'Filter']
E.columns = ['DATETIME', 'Zn', 'ntu', 'Zn_dr', 'Filter']
F.columns = ['DATETIME', 'Zn', 'ntu', 'Zn_dr', 'Filter']

AtoF = A.append(B, ignore_index =True).append(C, ignore_index = True).append(D, ignore_index=True).append(E, ignore_index=True).append(F, ignore_index=True)
AtoF = A.append(B, ignore_index =True).append(C, ignore_index = True).append(D, ignore_index=True).append(E, ignore_index=True).append(F, ignore_index=True)
AtoF.dropna(how='any', inplace=True)
AtoF.columns = ['DATETIME', 'Zn', 'ntu', 'Zn_dr', 'Filter']


#
#
#Aug2020_FT01AtoF_df
figPolishing = px.scatter(AtoF, x="DATETIME", y="Zn_dr", size="Zn", color="Filter", hover_name="ntu", size_max=40)
title_plotPolishing = "<b>Polishing Filter</b>" + '<br>' + 'Marker Size = Zn conc' + ' ; ' + 'On Hover = NTU'
figPolishing.update_layout(title_text=title_plotPolishing)
#
#-------------------------------------------------------------------------End----------------------------------------------------------------------------
plt.close()

#----------------------------------------------------------------------  NTRL----------------------------------------------------------------------------
#
#
figNTRL = make_subplots(specs=[[{"secondary_y": True}]])
figNTRL_axis_truefalse_list = [False, False, False, True, False,
                               True, False, False, False, True,
                               True, False, True, False, False, 
                               False]
generate_plot_from_df(figNTRL, Aug2020_ntrl_df, figNTRL_axis_truefalse_list)

#title_plot = 'Zn Drop Trend InterActive \n' + '\n Updated as of: ' + str(datetime.datetime.now())
t = datetime.datetime.now()
title_plotNTRL = "<b>NTRL Laboratory Analysis Results</b>" + '<br>' + 'Updated as of:' + str(t.strftime("%m/%d/%Y %H:%M:%S"))+'H'
# Add figure title
figNTRL.update_layout(title_text=title_plotNTRL)
# Set x-axis title
figNTRL.update_xaxes(title_text="DateTime")
# Set y-axes titles
figNTRL.update_yaxes(title_text="<b>Pb, Zn , Cu, Co, Cr, Ca, Si, Fe2</b>", secondary_y=False)
figNTRL.update_yaxes(title_text="<b>Ni, Mn, Al, Mg, pH, ORP</b>", secondary_y=True)
#
#-------------------------------------------------------------------------End----------------------------------------------------------------------------

plt.close()

#---------------------------------------------------------------------- 105TK03 -------------------------------------------------------------------------
#
#
figDeZnTK03 = make_subplots(specs=[[{"secondary_y": True}]])
figDeZNTK03_axis_truefalse_list = [True, False, False, True, True,
                                   False, True, False, False, False,
                                   False]
generate_plot_from_df(figDeZnTK03, Aug2020_TK03_df, figDeZNTK03_axis_truefalse_list)

#title_plot = 'Zn Drop Trend InterActive \n' + '\n Updated as of: ' + str(datetime.datetime.now())
title_plotDeZnTK03 = "<b>105TK03 Laboratory Analysis Results</b>" + '<br>' + 'Updated as of:' + str(t.strftime("%m/%d/%Y %H:%M:%S"))+'H'
# Add figure title and label
figDeZnTK03.update_layout(title_text=title_plotDeZnTK03)
figDeZnTK03.update_xaxes(title_text="DateTime") # Set x-axis title
figDeZnTK03.update_yaxes(title_text="TK03_Zn, TK05_Zn , 106TK01_Zn, 106TK01_Zn/Ni, VE04_Zn%, 106TH01_Zn%, Ni_Loss", secondary_y=False) # Set y-axes titles primary axis
figDeZnTK03.update_yaxes(title_text="Zn_ChangeRatio, 106TK01_Zn_DropRate, 104PU01_Ni, Mg, 105TK05_Ni, Ni_Loss_TK05", secondary_y=True) # Set y-axes titles secondary axis
#
#-------------------------------------------------------------------------End----------------------------------------------------------------------------
plt.close()

#---------  105TK05
figDeZnTK05 = make_subplots(specs=[[{"secondary_y": True}]])
figDeZnTK05_axis_truefalse_list = [False, False, False, True, True, 
                            True, False, False, False, True, 
                            True, True, False, True, True]
generate_plot_from_df(figDeZnTK05, Aug2020_TK05_df, figDeZnTK05_axis_truefalse_list)

#title_plot = 'Zn Drop Trend InterActive \n' + '\n Updated as of: ' + str(datetime.datetime.now())
title_plotDeZnTK05 = "<b>105TK05 Laboratory Analysis Results</b>" + '<br>' + 'Updated as of:' + str(t.strftime("%m/%d/%Y %H:%M:%S"))+'H'
# Add figure title and label
figDeZnTK05.update_layout(title_text=title_plotDeZnTK05)
figDeZnTK05.update_xaxes(title_text="DateTime") # Set x-axis title
figDeZnTK05.update_yaxes(title_text="<b>Pb, Zn , Cu, Co, Cr, Ca, Si, Fe2</b>", secondary_y=False) # Set y-axes titles primary axis
figDeZnTK05.update_yaxes(title_text="<b>Ni, Mn, Al, Mg, pH, ORP</b>", secondary_y=True) # Set y-axes titles secondary axis

#-------------------------------------------------------------------------End----------------------------------------------------------------------------
plt.close()
#---------  106TK01
figMSTK01 = make_subplots(specs=[[{"secondary_y": True}]])
figMSTK01_axis_truefalse_list = [False, False, False, True, True, 
                            True, True, True, True, True, 
                            True, True, True, False, True]
generate_plot_from_df(figMSTK01, Aug2020_MSTK01_df, figMSTK01_axis_truefalse_list)

#title_plot = 'Zn Drop Trend InterActive \n' + '\n Updated as of: ' + str(datetime.datetime.now())
title_plotMSTK01 = "<b>106TK01 Laboratory Analysis Results</b>" + '<br>' + 'Updated as of:' + str(t.strftime("%m/%d/%Y %H:%M:%S"))+'H'
# Add figure title and label
figMSTK01.update_layout(title_text=title_plotMSTK01)
figMSTK01.update_xaxes(title_text="DateTime") # Set x-axis title
figMSTK01.update_yaxes(title_text="Pb, Zn , Cu, Co, Cr, Ca, Si, Fe, ZnNi", secondary_y=False) # Set y-axes titles primary axis
figMSTK01.update_yaxes(title_text="Ni, Mg, Al, Mn, pH, ORP", secondary_y=True) # Set y-axes titles secondary axis

#-------------------------------------------------------------------------End----------------------------------------------------------------------------
plt.close()
#--------------------------------------------------------------------   106VE04  ------------------------------------------------------------------------
#
#
figMSVE04 = make_subplots(specs=[[{"secondary_y": True}]])
figMSVE04_axis_truefalse_list = [True, False, False, False, True, 
                            True, True, False, False, False, 
                            False, False, False, False, True, 
                            False]
generate_plot_from_df(figMSVE04, Aug2020_MSVE04_df, figMSVE04_axis_truefalse_list)

title_plotMSVE04 = "<b>VE04_Ni Trend InterActive</b>" + '<br>' + 'Updated as of:' + str(t.strftime("%m/%d/%Y %H:%M:%S"))+'H'
figMSVE04.update_layout(title_text=title_plotMSVE04) # Adds figure title
figMSVE04.update_xaxes(title_text="DateTime") # Set x-axis title
figMSVE04.update_yaxes(title_text="<b>Predicted</b> VE04 Ni", secondary_y=False) # Set y-axes titles primary axis
figMSVE04.update_yaxes(title_text="<b>Actual</b> VE04 Ni", secondary_y=True)    # Set y-axes titles secondary axis
#
#-------------------------------------------------------------------------End----------------------------------------------------------------------------

#--------------------------------------------------------------------   106VE04  ------------------------------------------------------------------------
#

#
figMSVE04_PRED = make_subplots(specs=[[{"secondary_y": True}]])
figMSVE04_axis_truefalse_list = [False, False]
generate_plot_from_df(figMSVE04_PRED, df_VE04Ni_MSpi[['VE04_Ni_Soln_pred', '106VE04_Ni_soln']], figMSVE04_axis_truefalse_list)
figMSVE04_PRED.add_trace(go.Bar(x=df_VE04Ni_MSpi.index, y=df_VE04Ni_MSpi['error'], name='error'), secondary_y=True)

title_plotMSVE04 = "<b>VE04_Ni Trend InterActive</b>" + '<br>' + 'Updated as of:' + str(t.strftime("%m/%d/%Y %H:%M:%S"))+'H'
figMSVE04_PRED.update_layout(title_text=title_plotMSVE04) # Adds figure title
figMSVE04_PRED.update_xaxes(title_text="DateTime") # Set x-axis title
figMSVE04_PRED.update_yaxes(title_text="<b>Predicted</b> VE04 Ni", secondary_y=False) # Set y-axes titles primary axis
figMSVE04_PRED.update_yaxes(title_text="<b>Actual</b> VE04 Ni", secondary_y=True)    # Set y-axes titles secondary axis
#
#-------------------------------------------------------------------------End----------------------------------------------------------------------------
plt.close()



with open(r'\\THGOBNA002\thpal\05 Production\150 DCS\Miscellaneous\InterActive Plots Aug2020.html', 'w') as f:
    f.write(figMSVE04_PRED.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(figNTRL.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(figDeZnTK03.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(figDeZnTK05.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(figMSTK01.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(figMSVE04.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(figPolishing.to_html(full_html=False, include_plotlyjs='cdn'))
   

